In [12]:
import re
import pandas as pd
import datetime

# Helper Functions
def find_res(text):
    """Extract RES values (e.g., 1378456) from the text."""
    if isinstance(text, str):
        match = re.search(r'(\D|\b)(\d{7})(\b|\D)', text)
        return match.group(2) if match else None
    return None

def convert_to_date(value):
    """Convert Excel-style numeric values or valid strings to 'dd/mm/yyyy'."""
    try:
        # Check if the value is a float or integer (potential Excel numeric date)
        if isinstance(value, (int, float)):
            # Convert using Excel's 1900 date system (adjust for offset)
            base_date = pd.Timestamp('1899-12-30')  # Use pandas Timestamp
            converted_date = base_date + pd.to_timedelta(value, unit='D')
            return converted_date.strftime("%d/%m/%Y")
        else:
            # Try to parse as a regular datetime string
            date = pd.to_datetime(value, dayfirst=True, errors='coerce')  # Specify dayfirst=True
            if pd.notnull(date):
                return date.strftime("%d/%m/%Y")
    except Exception as e:
        print(f"Error converting {value}, of type {type(value)} to date: {e}")
        return None
    return None


def reverse_date_format(date_str):
    """Reverse 'dd/mm/yyyy' to 'mm/dd/yyyy'."""
    try:
        day, month, year = date_str.split('/')
        return f"{month}/{day}/{year}"
    except (ValueError, AttributeError):
        return None

def normalize_polish_characters(value):
    """Replaces Polish diacritical characters with their ASCII equivalents."""
    polish_to_ascii = {
        "Ą": "A", "Ć": "C", "Ę": "E", "Ł": "L", "Ń": "N",
        "Ó": "O", "Ś": "S", "Ź": "Z", "Ż": "Z",
        "ą": "a", "ć": "c", "ę": "e", "ł": "l", "ń": "n",
        "ó": "o", "ś": "s", "ź": "z", "ż": "z"
    }
    if isinstance(value, str):
        for char, ascii_char in polish_to_ascii.items():
            value = value.replace(char, ascii_char)
    return value

# Matching Functions
def one_to_one(df1, df2, cols, result_df):
    """Performs one-to-one matching."""
    for _, row in df2.iterrows():
        match_filter = (df1[cols['df1']] == row[cols['df2']]) & (df1['kwota'] == row['kwota'])
        matching_rows = df1[match_filter]

        if not matching_rows.empty:
            result_df = pd.concat([
                result_df,
                pd.DataFrame({
                    "Sum of Amounts": [row["kwota"] / 100],
                    "List of Amounts": [[row["kwota"] / 100]],
                    "Title": matching_rows[cols['df1']].tolist(),
                    "Title2": [row[cols['df2']]],
                    "List of REZ": [matching_rows["rez"].tolist()]
                })
            ], ignore_index=True)
            df1.drop(matching_rows.index, inplace=True)
            df2.drop(row.name, inplace=True)
    return df1, df2, result_df

def one_to_many(df1, df2, cols, result_df):
    """Performs one-to-many matching."""
    for _, row in df2.iterrows():
        matching_rows = df1[df1[cols['df1']] == row[cols['df2']]]
        if not matching_rows.empty:
            sum_kwota = matching_rows['kwota'].sum()
            if sum_kwota == row['kwota']:
                result_df = pd.concat([
                    result_df,
                    pd.DataFrame({
                        "Sum of Amounts": [sum_kwota / 100],
                        "List of Amounts": [[
                            [amount / 100 for amount in matching_rows["kwota"].tolist()],
                            [row["kwota"] / 100]
                        ]],
                        "Title": [matching_rows[cols['df1']].tolist()],
                        "Title2": [row[cols['df2']]],
                        "List of REZ": [matching_rows["rez"].tolist()]
                    })
                ], ignore_index=True)
                df1.drop(matching_rows.index, inplace=True)
                df2.drop(row.name, inplace=True)
    return df1, df2, result_df

def many_to_many(df1, df2, group_col, result_df):
    """Performs many-to-many matching."""
    grouped_df1 = df1.groupby(group_col)['kwota'].sum().reset_index()
    grouped_df2 = df2.groupby(group_col)['kwota'].sum().reset_index()
    # make sure the columns have the same type
    
    merged_groups = pd.merge(grouped_df1, grouped_df2, on=group_col, suffixes=('_df1', '_df2'))
    
    for _, row in merged_groups.iterrows():
        if row['kwota_df1'] == row['kwota_df2']:
            matching_rows_df1 = df1[df1[group_col] == row[group_col]]
            matching_rows_df2 = df2[df2[group_col] == row[group_col]]

            result_df = pd.concat([
                result_df,
                pd.DataFrame({
                    "Sum of Amounts": [row['kwota_df1'] / 100],
                    "List of Amounts": [[
                        [amount / 100 for amount in matching_rows_df1['kwota']],
                        [amount / 100 for amount in matching_rows_df2['kwota']]
                    ]],
                    "Title": [matching_rows_df1[group_col].tolist()],
                    "Title2": [matching_rows_df2[group_col].tolist()],
                    "List of REZ": [matching_rows_df1["rez"].tolist()]
                })
            ], ignore_index=True)
            df1.drop(matching_rows_df1.index, inplace=True)
            df2.drop(matching_rows_df2.index, inplace=True)
    return df1, df2, result_df

# Main Logic
# Load data and preprocess
odeme = pd.read_excel("0511.xlsx", sheet_name="odeme")
bank = pd.read_excel("0511.xlsx", sheet_name="bank")

# Convert 'kwota' to integer
odeme['kwota'] = odeme['kwota'] * 100
bank['kwota'] = bank['kwota'] * -100
odeme['kwota'] = odeme['kwota'].astype(int)
bank['kwota'] = bank['kwota'].astype(int)

# Normalize text and add `mapped_tytul`
odeme["mapped_tytul"] = odeme["tytul"].apply(normalize_polish_characters)
bank["mapped_tytul"] = bank["tytul"].apply(normalize_polish_characters)

odeme['mapped_tytul'] = odeme['mapped_tytul'].apply(lambda x: x.strip() if isinstance(x, str) else x)
bank['mapped_tytul'] = bank['mapped_tytul'].apply(lambda x: x.strip() if isinstance(x, str) else x)

# Prepare date columns
bank['mapped_dates'] = bank['mapped_tytul'].apply(convert_to_date)
bank['mapped_dates_reversed'] = bank['mapped_dates'].apply(reverse_date_format)

odeme['mapped_dates'] = odeme['mapped_tytul'].apply(convert_to_date)
odeme['mapped_dates_reversed'] = odeme['mapped_dates'].apply(reverse_date_format)

odeme["mapped_tytul"] = odeme["mapped_tytul"].astype(str)
bank["mapped_tytul"] = bank["mapped_tytul"].astype(str)

# Extract RES values
odeme['res_group'] = odeme['mapped_tytul'].apply(find_res)
bank['res_group'] = bank['mapped_tytul'].apply(find_res)
odeme['res_group2'] = odeme['rez'].apply(find_res)

# Initialize result dataframe
matched_lines = pd.DataFrame(columns=["Sum of Amounts", "List of Amounts", "Title", "Title2", "List of REZ"])
unmatched_odeme = odeme.copy()
unmatched_bank = bank.copy()

unmatched_odeme, unmatched_bank, matched_lines = one_to_one(unmatched_odeme, unmatched_bank, cols={'df1': 'res_group', 'df2': 'res_group'}, result_df=matched_lines)
unmatched_odeme, unmatched_bank, matched_lines = one_to_many(unmatched_odeme, unmatched_bank, cols={'df1': 'res_group', 'df2': 'res_group'}, result_df=matched_lines)
unmatched_odeme, unmatched_bank, matched_lines = many_to_many(unmatched_odeme, unmatched_bank, group_col='res_group', result_df=matched_lines)

unmatched_odeme, unmatched_bank, matched_lines = one_to_one(unmatched_odeme, unmatched_bank, cols={'df1': 'res_group2', 'df2': 'res_group'}, result_df=matched_lines)
unmatched_odeme, unmatched_bank, matched_lines = one_to_many(unmatched_odeme, unmatched_bank, cols={'df1': 'res_group2', 'df2': 'res_group'}, result_df=matched_lines)
unmatched_odeme, unmatched_bank, matched_lines = many_to_many(unmatched_odeme, unmatched_bank, group_col='res_group', result_df=matched_lines)

# Perform matching step-by-step
for column in ['tytul', 'mapped_tytul', 'mapped_dates', 'mapped_dates_reversed']:
    for column2 in ['tytul', 'mapped_tytul', 'mapped_dates', 'mapped_dates_reversed']:
        unmatched_odeme, unmatched_bank, matched_lines = one_to_one(
            unmatched_odeme, unmatched_bank, cols={'df1': column, 'df2': column2}, result_df=matched_lines)
        unmatched_odeme, unmatched_bank, matched_lines = one_to_many(
            unmatched_odeme, unmatched_bank, cols={'df1': column, 'df2': column2}, result_df=matched_lines)
        unmatched_odeme, unmatched_bank, matched_lines = many_to_many(
            unmatched_odeme, unmatched_bank, group_col=column2, result_df=matched_lines)
        
# divide by 100 to get the original amount
unmatched_odeme['kwota'] = unmatched_odeme['kwota'] / 100
unmatched_bank['kwota'] = unmatched_bank['kwota'] / 100

# Save results
output_path = "odeme2911.xlsx"
with pd.ExcelWriter(output_path) as writer:
    matched_lines.to_excel(writer, sheet_name="Dopasowane", index=False)
    unmatched_odeme.to_excel(writer, sheet_name="Odeme - niedopasowane", index=False)
    unmatched_bank.to_excel(writer, sheet_name="Bank - niedopasowane", index=False)

print(f"Results saved to {output_path}")


Results saved to odeme2911.xlsx


C:\Users\Kamil Tomaszek\AppData\Local\Temp\ipykernel_33228\3170795930.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([
